<a href="https://colab.research.google.com/github/AbhinavKSeth/SRE/blob/master/Attempt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import PIL
import copy
import time
import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
from skimage import io, transform
from torchvision import datasets
import matplotlib.image as mpimg
from torchvision import transforms
from torch.optim import lr_scheduler
import torchvision.models as models
from torch.utils.data import Dataset

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine([-90,90]),
        transforms.ToTensor()
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor()
        # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [0]:
data_dir = "/content/drive/My Drive/path/images"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
image_datasets['val'].class_to_idx

{'Defective': 0, 'Good': 1}

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    # if(phase == 'val'): 
                    #   print(labels)
                    #   print(outputs)

                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            # print(labels)
            # print(outputs)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
vgg19 = models.vgg19(pretrained=True)

for param in vgg19.parameters():
    param.requires_grad_(False)
vgg19.classifier = nn.Sequential(
                    nn.Linear(25088,4096,bias = True),
                    nn.ReLU(inplace=True),
                    nn.Dropout(p=0.5, inplace=False),
                    nn.Linear(4096,2048,bias = True),
                    nn.ReLU(inplace=True),
                    nn.Dropout(p=0.5, inplace=False),
                    nn.Linear(2048,2,bias = True)
                    )
vgg19  = vgg19.to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth


In [0]:
criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer1 = optim.SGD(vgg19.classifier.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer1, step_size=7, gamma=0.1)

In [0]:
vgg19 = train_model(vgg19, criterion, optimizer1,
                         exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.5773 Acc: 0.7038
val Loss: 0.5201 Acc: 0.7530

Epoch 1/24
----------
train Loss: 0.5300 Acc: 0.7475
val Loss: 0.5054 Acc: 0.7774

Epoch 2/24
----------
train Loss: 0.5010 Acc: 0.7561
val Loss: 0.5102 Acc: 0.7271

Epoch 3/24
----------
train Loss: 0.4897 Acc: 0.7622
val Loss: 0.4848 Acc: 0.7729

Epoch 4/24
----------
train Loss: 0.4800 Acc: 0.7683
val Loss: 0.4939 Acc: 0.7485

Epoch 5/24
----------
train Loss: 0.4711 Acc: 0.7805
val Loss: 0.4537 Acc: 0.7957

Epoch 6/24
----------
train Loss: 0.4589 Acc: 0.7912
val Loss: 0.4938 Acc: 0.7439

Epoch 7/24
----------
train Loss: 0.4465 Acc: 0.7866
val Loss: 0.4678 Acc: 0.7622

Epoch 8/24
----------
train Loss: 0.4330 Acc: 0.8018
val Loss: 0.4591 Acc: 0.7805

Epoch 9/24
----------
train Loss: 0.4248 Acc: 0.7983
val Loss: 0.4609 Acc: 0.7683

Epoch 10/24
----------
train Loss: 0.4374 Acc: 0.7932
val Loss: 0.4527 Acc: 0.7912

Epoch 11/24
----------
train Loss: 0.4247 Acc: 0.7957
val Loss: 0.4587 Acc: 0.7774

Ep